In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('seaborn')
plt.rcParams['figure.figsize'] = [20, 7]

In [2]:
confirmed = pd.read_csv('CSSEGISandData/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')

In [3]:
deaths = pd.read_csv('CSSEGISandData/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

In [4]:
recovered = pd.read_csv('CSSEGISandData/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

In [14]:
confirmed

,province,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,Anhui,Mainland China,1,9,15,39,60,70,106,152,...,989,989,989,990,990,990,990,990,990,990
1,Beijing,Mainland China,14,22,36,41,68,80,91,111,...,400,400,410,410,411,413,414,414,418,418
2,Chongqing,Mainland China,6,9,27,57,75,110,132,147,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,1,5,10,18,35,59,80,84,...,294,294,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,0,2,2,4,7,14,19,24,...,91,91,91,91,91,91,91,91,91,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,"Fort Bend County, TX",US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
169,"Grant County, WA",US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
170,"Queens County, NY",US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
171,"Santa Rosa County, FL",US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
confirmed.drop(['Lat', 'Long'], axis=1, inplace=True)

In [12]:
recovered.drop(['Lat', 'Long'], axis=1, inplace=True)

In [13]:
deaths.drop(['Lat', 'Long'], axis=1, inplace=True)

In [7]:
recovered.rename({'Province/State': 'province', 'Country/Region': 'country'}, axis=1, inplace=True)

In [7]:
confirmed.rename({'Province/State': 'province', 'Country/Region': 'country'}, axis=1, inplace=True)

In [8]:
deaths.rename({'Province/State': 'province', 'Country/Region': 'country'}, axis=1, inplace=True)

In [10]:
confirmed[~confirmed['province'].isnull()].country.value_counts()

US                44
Mainland China    31
Australia          8
Canada             4
Macau              1
Hong Kong          1
Others             1
Taiwan             1
Name: country, dtype: int64

In [18]:
df = confirmed.copy()

In [34]:
df_uc = df[((df.country == 'Mainland China') |  (df.country == 'US'))]

In [35]:
df_no_uc = df[~((df.country == 'Mainland China') |  (df.country == 'US'))]

In [36]:
df_uc[~df_uc['province'].isnull()].country.value_counts()

US                44
Mainland China    31
Name: country, dtype: int64

In [38]:
df_united = df_no_uc.groupby('country').sum()

In [105]:
def removeSome(df):
    df = df.copy()
    df = df[~df.country.str.contains('Others')]
    df_no_uc = df[~((df.country == 'Mainland China') |  (df.country == 'US'))]
    df_united = df_no_uc.groupby('country').sum()
    df_united.index.rename('place', inplace=True)
    df_uc = df[((df.country == 'Mainland China') |  (df.country == 'US'))]
    df_uc = df_uc[~df_uc.province.str.contains('Diamond Princess')]
    df_uc['place'] = df_uc['country'] + '/' + df_uc.province.apply(lambda x: x.split(', ')[-1])
    df_uc.drop(['province', 'country'], axis=1, inplace=True)
    df_uc.set_index('place', inplace=True)
    return pd.concat([df_united, df_uc])
con = removeSome(confirmed)
rec = removeSome(recovered)
dea = removeSome(deaths)